# Helper Functions
1. Aim to reduce the training data memory and could possibily achieve 16x memory reduction
2. Many people store the article_id as a string to keep the leading zero. However a string of length 10 uses 10 bytes. It is better to convert this column to int32 which only uses 4 bytes. Then do all your processing, i.e. train.groupby('article_id') etc etc. Then before writing submission, just convert it back

In [1]:
from tqdm import tqdm
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

In [2]:
# Source -> Memory Trick! -> https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635

def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    # convert the id from str to id
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)

We want this to be fast. I can get as much RAM as I will ever need through VMs on GCP, but that is not the point. I want to see how far I can push my local hardware, but this goes even beyond that.

I need the speed to make a good use of my time as I continue to build my understanding of what RecSys models are about. And the path to this leads through making the data I will work on smaller.

# Import Data

In [3]:
%%time
import pandas as pd

transactions = pd.read_csv('/Users/gloriahwoang/Desktop/DS_Portfolio/hmRecommenderSys/transactions_train.csv', dtype={"article_id": "str"})
customers = pd.read_csv('/Users/gloriahwoang/Desktop/DS_Portfolio/hmRecommenderSys/customers.csv')
articles = pd.read_csv('/Users/gloriahwoang/Desktop/DS_Portfolio/hmRecommenderSys/articles.csv', dtype={"article_id": "str"})

CPU times: user 19.4 s, sys: 4.1 s, total: 23.5 s
Wall time: 40.4 s


# Data conversion to save more memeory

## Transactions
1. article_id:  a string of length 10 uses 10 bytes. It is better to convert this column to int32 which only uses 4 bytes. Before writing a submission, just convert it back
2. customer_id: convert string to int64
3. t_date: reformat to datetime ‘%Y-%m-%d’ 
4. sales_channel_id: convert to int8
5. prices: convert to float32
6. week: new column which is calculated from t_date, stored as int8

In [4]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id         3846387204
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [5]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 8.0 GB


In [6]:
%%time
transactions['customer_id'].nunique()

CPU times: user 4.48 s, sys: 364 ms, total: 4.84 s
Wall time: 7.45 s


1362281

In [7]:
%%time
# if we convert the id from string to int we will saved 50% of memory
transactions['customer_id'] = customer_hex_id_to_int(transactions['customer_id'])
transactions['customer_id'].nunique()

CPU times: user 11.7 s, sys: 3.45 s, total: 15.1 s
Wall time: 25 s


1362281

In [8]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id          254306592
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [9]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       uint64 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 4.7 GB


In [10]:
%%time

transactions.t_dat = pd.to_datetime(transactions.t_dat, format='%Y-%m-%d')

CPU times: user 1.68 s, sys: 261 ms, total: 1.94 s
Wall time: 3.05 s


In [11]:
transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

In [12]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
 5   week              int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1), uint64(1)
memory usage: 3.2 GB


Let's do something about the `article_id` (both here and on `articles`) and let's take a closer look at `price`, `sales_channel_id` and `week`.

In [13]:
transactions.article_id = article_id_str_to_int(transactions.article_id)
articles.article_id = article_id_str_to_int(articles.article_id)

transactions.week = transactions.week.astype('int8')
transactions.sales_channel_id = transactions.sales_channel_id.astype('int8')
transactions.price = transactions.price.astype('float32')

In [14]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        int32         
 3   price             float32       
 4   sales_channel_id  int8          
 5   week              int8          
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2), uint64(1)
memory usage: 788.2 MB


In [15]:
transactions.drop(columns='t_dat').info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   customer_id       uint64 
 1   article_id        int32  
 2   price             float32
 3   sales_channel_id  int8   
 4   week              int8   
dtypes: float32(1), int32(1), int8(2), uint64(1)
memory usage: 545.7 MB


## Customer
1. customer_id: convert string to int64
2. FN, Active, age: fill NA with -1 and convert to int8
3. club_member_status, postal_code, fashion_news_frequency: categorizing data with int8

In [16]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 512.3 MB


In [17]:
articles.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int32 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

Well, this stuff will be getting merged with our transactions df at some point, so I guess we can also make this smaller and easier to work with down the road.

In [18]:
customers['club_member_status'].unique()

array(['ACTIVE', nan, 'PRE-CREATE', 'LEFT CLUB'], dtype=object)

In [19]:
customers.customer_id = customer_hex_id_to_int(customers.customer_id)
for col in ['FN', 'Active', 'age']:
    customers[col].fillna(-1, inplace=True)
    customers[col] = customers[col].astype('int8')

In [20]:
#categorize the data
customers.club_member_status = Categorize().fit_transform(customers[['club_member_status']]).club_member_status
customers.postal_code = Categorize().fit_transform(customers[['postal_code']]).postal_code
customers.fashion_news_frequency = Categorize().fit_transform(customers[['fashion_news_frequency']]).fashion_news_frequency

In [21]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   customer_id             1371980 non-null  uint64
 1   FN                      1371980 non-null  int8  
 2   Active                  1371980 non-null  int8  
 3   club_member_status      1371980 non-null  int8  
 4   fashion_news_frequency  1371980 non-null  int8  
 5   age                     1371980 non-null  int8  
 6   postal_code             1371980 non-null  int32 
dtypes: int32(1), int8(5), uint64(1)
memory usage: 22.2 MB


In [22]:
customers

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,6883939031699146327,-1,-1,0,0,49,6305
1,11246327431398957306,-1,-1,0,0,25,33726
2,18439897732908966680,-1,-1,0,0,24,3247
3,18352672461570950206,-1,-1,0,0,54,168643
4,18162778555210377306,1,1,0,1,52,168645
...,...,...,...,...,...,...,...
1371975,7551062398649767985,-1,-1,0,0,24,50351
1371976,9305341941720086711,-1,-1,0,0,21,80169
1371977,10160427316885688932,1,1,0,1,21,106737
1371978,2551401172826382186,1,1,0,1,18,111894


## Articles
1. convert all int64 columns to int32
2. keep all other features 

In [25]:
articles.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int32 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [26]:
for col in articles.columns:
    if articles[col].dtype == 'int64':
        articles[col] = articles[col].astype('int32')

# Write back to disk
And this concludes our raw data preparation step! Let's now write everything back to disk.

In [27]:
transactions.sort_values(['t_dat', 'customer_id'], inplace=True)

In [29]:
#further save memory to store the data as parquet format
%time

transactions.to_parquet('../data/transactions_train.parquet')
customers.to_parquet('../data/customers.parquet')
articles.to_parquet('../data/articles.parquet')

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs
